In [1]:
import geopandas as gpd
import pandas as pd
from requests.auth import HTTPBasicAuth
import requests
import json
import plotly.express as px
import os
import rtree

In [2]:
# fewsnet_all = gpd.read_file("./data/FEWS_NET_LH_World.zip")
# fewsnet_all = fewsnet_all.to_crs(epsg=4326)
# fewsnet_all.head(1)

In [3]:
lhz = gpd.read_file("./data/FEWS_NET_LH_World.zip")
lhz = lhz.to_crs(epsg=4326)
lhz

,OBJECTID,FNID,EFF_YEAR,COUNTRY,LZNUM,LZCODE,LZNAMEEN,LZNAMEFR,LZNAMESP,LZNAMEPT,CLASS,LZSZCODE,LZTYPE,MAINCROPS,MAINLSTOCK,geometry
0,30,AO2013L102,2013,AO,2,AO02,Transitional banana and pineapple farming,None,None,Transicional do cultivo da banana e ananás,AS09,None,None,None,None,"POLYGON ((14.32935 -10.30132, 14.35352 -10.313..."
1,31,AO2013L104,2013,AO,4,AO04,Sub humid livestock and maize,None,None,Sub húmida de pastorícia e milho,PS03,None,None,None,None,"POLYGON ((17.89424 -13.67606, 17.90491 -13.695..."
2,32,AO2013L105,2013,AO,5,AO05,Mid-eastern cassava and savannah forest,None,None,Centro-este de mandioca e floresta de savana,AG04,None,None,None,None,"POLYGON ((17.92018 -11.52576, 17.93213 -11.527..."
3,33,AO2013L108,2013,AO,8,AO08,"Transitional lowland maize, cassava, and beans",None,None,"Baixa transicional de milho, mandioca e feijões",AS05,None,None,None,None,"POLYGON ((13.83407 -7.65854, 13.83546 -7.68132..."
4,34,AO2013L110,2013,AO,10,AO10,Savannah forest and market-oriented cassava,None,None,Floresta de savana e mandioca orientada ao mer...,AN08,None,None,None,None,"POLYGON ((17.11363 -7.42542, 17.12791 -7.42627..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,0,TD2021L104,2021,TD,4,TD04,Southeast Flood Retreat and Gum Arabic,Zone sud-est: cultures de décrue et gomme arab...,None,None,AN01,None,None,None,None,"POLYGON ((22.47709 12.01926, 22.47845 12.01923..."
861,0,TD2021L103,2021,TD,3,TD03,South-central Cereals,Zone sud-centrale céréaliculture,None,None,AN04,None,None,None,None,"POLYGON ((19.40499 11.10400, 19.47900 11.04799..."
862,0,TD2021L102,2021,TD,2,TD02,Southwest Rice,Zone sud-ouest de riz \r,None,None,AS05,None,None,None,None,"POLYGON ((15.39857 9.77331, 15.39858 9.77331, ..."
863,0,TD2021L105,2021,TD,5,TD05,Central Agropastoral,Zone centrale agropastorale,None,None,PS03,None,None,None,None,"POLYGON ((17.27100 13.66200, 17.23400 13.42700..."


In [4]:
adm1 = gpd.read_file("./data/FEWSNET_World_Admin/FEWSNET_Admin1.shp")
adm1 = adm1.to_crs(epsg=4326)
adm1

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ALIASES,geometry
0,AO2008A101,2008,AO,Angola,Bengo,None,None,Bengo,None,"POLYGON ((13.99846 -7.65368, 13.99921 -7.65384..."
1,AO2008A102,2008,AO,Angola,Benguela,None,None,Benguela,None,"POLYGON ((13.87370 -11.76882, 13.87458 -11.768..."
2,AO2008A103,2008,AO,Angola,Bie,None,None,Bie,None,"POLYGON ((16.61016 -10.57761, 16.61522 -10.578..."
3,AO2008A104,2008,AO,Angola,Cabinda,None,None,Cabinda,None,"POLYGON ((12.82282 -4.40323, 12.83247 -4.40642..."
4,AO2008A105,2008,AO,Angola,Cunene,None,None,Cunene,None,"POLYGON ((16.54543 -15.13035, 16.54644 -15.130..."
...,...,...,...,...,...,...,...,...,...,...
664,AF2017A122,2017,AF,Afghanistan,Sari Pul,None,None,None,~Sar-e-Pul~Sar-e Pul~,"POLYGON ((66.00733 36.50566, 66.00871 36.50549..."
665,AF2017A118,2017,AF,Afghanistan,Takhar,None,None,None,~Tekhar~Taluqan~Talukan~,"MULTIPOLYGON (((70.06893 37.38423, 70.06896 37..."
666,AF2017A125,2017,AF,Afghanistan,Uruzgan,None,None,None,~Oruzgan~Owruzgan~Urozgan~Orozgan~,"POLYGON ((66.52982 33.59141, 66.53017 33.58970..."
667,AF2017A104,2017,AF,Afghanistan,Wardak,None,None,None,~Maydan~Vardak~Werdek~Wardag~,"POLYGON ((68.29716 34.78276, 68.29839 34.78109..."


In [5]:
adm2 = gpd.read_file("./data/FEWSNET_World_Admin/FEWSNET_Admin2.shp")
adm2 = adm2.to_crs(epsg=4326)
adm2

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry
0,AO2008A21701,2008,AO,Angola,Uige,None,None,Uige,Alto Cuale,None,None,Alto Cuale,None,"POLYGON ((16.34542 -7.47550, 16.34498 -7.47693..."
1,AO2008A21501,2008,AO,Angola,Moxico,None,None,Moxico,Alto Zambeze,None,None,Alto Zambeze,None,"POLYGON ((24.01645 -11.01787, 24.01387 -11.030..."
2,AO2008A20901,2008,AO,Angola,Kuanza Norte,None,None,Kuanza Norte,Ambaca,None,None,Ambaca,None,"POLYGON ((15.69911 -8.01768, 15.70128 -8.01780..."
3,AO2008A21001,2008,AO,Angola,Kuanza Sul,None,None,Kuanza Sul,Amboim,None,None,Amboim,None,"POLYGON ((14.51918 -10.76831, 14.52180 -10.770..."
4,AO2008A20101,2008,AO,Angola,Bengo,None,None,Bengo,Ambriz,None,None,Ambriz,None,"POLYGON ((13.81469 -7.66262, 13.81628 -7.66297..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,AF2017A20105,2017,AF,Afghanistan,Kabul,None,None,None,Deh Sabz,None,None,None,~Dehsabz~Deh-e-Sabz~Deh-e Sabz~,"POLYGON ((69.46155 34.58448, 69.46121 34.58369..."
6561,AF2017A20907,2017,AF,Afghanistan,Baghlan,None,None,None,Andarab,None,None,None,None,"POLYGON ((69.04545 35.81928, 69.04450 35.81818..."
6562,AF2017A21713,2017,AF,Afghanistan,Badakhshan,None,None,None,Shahr-e-Buzorg,None,None,None,~Shahr-e Buzurg~Shahri Buzurg~Shahr-e Buzurg~,"POLYGON ((70.06893 37.38423, 70.07026 37.38434..."
6563,AF2017A20102,2017,AF,Afghanistan,Kabul,None,None,None,Paghman,None,None,None,None,"POLYGON ((69.03724 34.59171, 69.03838 34.59173..."


In [6]:
minidf = gpd.read_file("./data/ALL_HFIC/East Africa/EA_201802_CS.shp")
minidf = minidf.to_crs(epsg=4326)
minidf.head(20)

,CS,HA0,geometry
0,1.0,0.0,"MULTIPOLYGON (((39.36784 -4.65351, 39.36945 -4..."
1,1.0,1.0,"MULTIPOLYGON (((42.78954 3.30069, 42.71921 3.2..."
2,2.0,0.0,"MULTIPOLYGON (((39.54055 -3.63975, 39.54312 -3..."
3,2.0,1.0,"MULTIPOLYGON (((44.45677 2.13468, 44.44286 2.1..."
4,3.0,0.0,"MULTIPOLYGON (((36.80593 -1.59263, 36.80575 -1..."
5,3.0,1.0,"MULTIPOLYGON (((31.37783 6.50423, 31.47383 6.3..."
6,4.0,0.0,"MULTIPOLYGON (((30.12183 6.35244, 30.15537 6.2..."
7,88.0,0.0,"MULTIPOLYGON (((29.67210 -0.88200, 29.67230 -0..."
8,99.0,0.0,"MULTIPOLYGON (((46.98970 8.00000, 47.01239 8.0..."


In [7]:
largedf = gpd.read_file("./data/ALL_HFIC/East Africa/EA_202110_CS.shp")
largedf = largedf.to_crs(epsg=4326)
largedf

,cov_start,cov_end,report_mon,country,unit_name,ADMIN0,ADMIN1,ADMIN2,ADMIN3,LZCODE,LZNAME,CS,HA0,geometry
0,10-2021,10-2021,10-2021,BI,"Buragane, Makamba, Burundi",Burundi,Makamba,None,None,BI01,Buragane,1,0,"POLYGON ((29.79122 -4.01030, 29.79211 -4.01100..."
1,10-2021,10-2021,10-2021,BI,"Buragane, Rutana, Burundi",Burundi,Rutana,None,None,BI01,Buragane,1,0,"POLYGON ((29.97427 -3.97512, 29.97411 -3.97570..."
2,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bubanza, Burundi",Burundi,Bubanza,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.39921 -2.96037, 29.39955 -2.96100..."
3,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bujumbura Rural, Burundi",Burundi,Bujumbura Rural,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.50249 -3.26355, 29.50681 -3.26968..."
4,10-2021,10-2021,10-2021,BI,"Congo Ridge Millet, Bururi, Burundi",Burundi,Bururi,None,None,BI02,Congo Ridge Millet,1,0,"POLYGON ((29.64906 -4.10597, 29.64796 -4.10726..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Arua, ...",Uganda,Northern,Arua,None,UG15,None,2,0,"MULTIPOLYGON (((31.20456 3.33572, 31.20450 3.3..."
2600,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Koboko...",Uganda,Northern,Koboko,None,UG15,None,2,0,"POLYGON ((31.10081 3.74161, 31.10047 3.73584, ..."
2601,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Madi O...",Uganda,Northern,Madi Okollo,None,UG15,None,2,0,"MULTIPOLYGON (((31.07543 2.82913, 31.07725 2.8..."
2602,10-2021,10-2021,10-2021,UG,"West Nile Tobacco Cassava Sorghum Zone, Marach...",Uganda,Northern,Maracha,None,UG15,None,2,0,"POLYGON ((31.04593 3.32229, 31.04595 3.32224, ..."


In [7]:
# goes through each folder and creates a dictionary with file name and column names
# there is also a list created with just the file names for use later
# It adds each of the files with more metadata to the large_files list

large_files = []

# Central America files
cam_dict = {}
cam_files = os.listdir("./data/ALL_HFIC/Central America and the Caribbean")
for file in cam_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Central America and the Caribbean/" + file)
        cam_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# Central Asia files
cas_dict = {}
cas_files = os.listdir("./data/ALL_HFIC/Central Asia")
for file in cas_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Central Asia/" + file)
        cas_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# East Africa files
ea_dict = {}
ea_files = os.listdir("./data/ALL_HFIC/East Africa")
for file in ea_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/East Africa/" + file)
        ea_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# West Africa files
wa_dict = {}
wa_files = os.listdir("./data/ALL_HFIC/West Africa")
for file in wa_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/West Africa/" + file)
        wa_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)

# South Africa files
sa_dict = {}
sa_files = os.listdir("./data/ALL_HFIC/Southern Africa")
for file in sa_files:
    if file.endswith(".shp"):
        tempgdf = gpd.read_file("./data/ALL_HFIC/Southern Africa/" + file)
        sa_dict[file] = tempgdf.columns
        if "ADMIN1" in tempgdf.columns:
            large_files.append(file)


In [9]:
large_files

['LAC_201107_CS.shp',
 'LAC_200910_CS.shp',
 'LAC_201204_CS.shp',
 'LAC_201304_CS.shp',
 'LAC_201410_CS.shp',
 'CAaC_202110_CS.shp',
 'EA_202110_CS.shp',
 'WA_202110_CS.shp',
 'SA_202110_CS.shp']

At this point we have dictionaries of files in each part of the world. There are files with more metadata for every region besides central asia.
There is one weird dataset in central asia, CA_201201_CS.shp, but it just has one extra column `CS201201_P` which is identical to the `CS` column so it shouldn't cause any issues.

In [8]:
# workshoping don't run
#     if file.endswith(".shp"):
#         tempgdf = gpd.read_file("./data/ALL_HFIC/Central America and the Caribbean/" + file)
#         tempgdf = tempgdf.to_crs(epsg=4326)
#         if "ADMIN1" in tempgdf.columns:
#             tempgdf = gpd.sjoin(tempgdf, adm1, how="left", op="intersects")
#             tempgdf.to_file("./data/ALL_HFIC/Central America and the Caribbean/" + file)
#         else:
#             tempgdf = gpd.sjoin(tempgdf, adm2, how="left", op="intersects")
#             tempgdf.to_file("./data/ALL_HFIC/Central America and the Caribbean/" + file)

In [9]:
join_left_df = adm2.sjoin(minidf, how="left")
join_left_df

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry,index_right,CS,HA0
0,AO2008A21701,2008,AO,Angola,Uige,None,None,Uige,Alto Cuale,None,None,Alto Cuale,None,"POLYGON ((16.34542 -7.47550, 16.34498 -7.47693...",NaN,NaN,NaN
1,AO2008A21501,2008,AO,Angola,Moxico,None,None,Moxico,Alto Zambeze,None,None,Alto Zambeze,None,"POLYGON ((24.01645 -11.01787, 24.01387 -11.030...",NaN,NaN,NaN
2,AO2008A20901,2008,AO,Angola,Kuanza Norte,None,None,Kuanza Norte,Ambaca,None,None,Ambaca,None,"POLYGON ((15.69911 -8.01768, 15.70128 -8.01780...",NaN,NaN,NaN
3,AO2008A21001,2008,AO,Angola,Kuanza Sul,None,None,Kuanza Sul,Amboim,None,None,Amboim,None,"POLYGON ((14.51918 -10.76831, 14.52180 -10.770...",NaN,NaN,NaN
4,AO2008A20101,2008,AO,Angola,Bengo,None,None,Bengo,Ambriz,None,None,Ambriz,None,"POLYGON ((13.81469 -7.66262, 13.81628 -7.66297...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,AF2017A20105,2017,AF,Afghanistan,Kabul,None,None,None,Deh Sabz,None,None,None,~Dehsabz~Deh-e-Sabz~Deh-e Sabz~,"POLYGON ((69.46155 34.58448, 69.46121 34.58369...",NaN,NaN,NaN
6561,AF2017A20907,2017,AF,Afghanistan,Baghlan,None,None,None,Andarab,None,None,None,None,"POLYGON ((69.04545 35.81928, 69.04450 35.81818...",NaN,NaN,NaN
6562,AF2017A21713,2017,AF,Afghanistan,Badakhshan,None,None,None,Shahr-e-Buzorg,None,None,None,~Shahr-e Buzurg~Shahri Buzurg~Shahr-e Buzurg~,"POLYGON ((70.06893 37.38423, 70.07026 37.38434...",NaN,NaN,NaN
6563,AF2017A20102,2017,AF,Afghanistan,Kabul,None,None,None,Paghman,None,None,None,None,"POLYGON ((69.03724 34.59171, 69.03838 34.59173...",NaN,NaN,NaN


In [10]:
# sort join_left_df by CS
join_left_df.sort_values(by="CS", ascending=False)

# if CS column is not null add row to new dataframe named scratch
scratch = join_left_df[join_left_df["CS"].notnull()]
scratch

,FNID,EFF_YEAR,COUNTRY,ADMIN0,ADMIN1,ADMIN1FR,ADMIN1SP,ADMIN1PT,ADMIN2,ADMIN2FR,ADMIN2SP,ADMIN2PT,ALIASES,geometry,index_right,CS,HA0
337,CF2003A25203,2003,CF,Central African Republic,Haute-Kotto,Haute-Kotto,None,None,Yalinga,Yalinga,None,None,None,"POLYGON ((23.50040 8.96243, 23.50361 8.96193, ...",6.0,4.0,0.0
337,CF2003A25203,2003,CF,Central African Republic,Haute-Kotto,Haute-Kotto,None,None,Yalinga,Yalinga,None,None,None,"POLYGON ((23.50040 8.96243, 23.50361 8.96193, ...",0.0,1.0,0.0
338,CF2003A25301,2003,CF,Central African Republic,Vakaga,Vakaga,None,None,Birao,Birao,None,None,None,"POLYGON ((22.52068 11.00757, 22.55266 10.99373...",0.0,1.0,0.0
350,CF2003A26301,2003,CF,Central African Republic,Haut-Mbomou,Haut-Mbomou,None,None,Obo,Obo,None,None,None,"POLYGON ((26.57464 6.02772, 26.58015 6.02655, ...",2.0,2.0,0.0
351,CF2003A26302,2003,CF,Central African Republic,Haut-Mbomou,Haut-Mbomou,None,None,Bambouti,Bambouti,None,None,None,"POLYGON ((26.80225 5.98314, 26.80225 5.98314, ...",2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4894,ET2014A20507,2014,ET,Ethiopia,Somali,None,None,None,Dollo,None,None,None,~Doollo~Werder~Warder~Werder~Wardier~,"POLYGON ((45.95040 5.91119, 45.94751 5.91260, ...",4.0,3.0,0.0
4894,ET2014A20507,2014,ET,Ethiopia,Somali,None,None,None,Dollo,None,None,None,~Doollo~Werder~Warder~Werder~Wardier~,"POLYGON ((45.95040 5.91119, 45.94751 5.91260, ...",5.0,3.0,1.0
4894,ET2014A20507,2014,ET,Ethiopia,Somali,None,None,None,Dollo,None,None,None,~Doollo~Werder~Warder~Werder~Wardier~,"POLYGON ((45.95040 5.91119, 45.94751 5.91260, ...",8.0,99.0,0.0
4895,ET2014A20406,2014,ET,Ethiopia,Oromia,None,None,None,North Shewa,None,None,None,~Semen Shewa~Semien Shewa~North Shoa~,"POLYGON ((38.84808 10.36458, 38.84691 10.36019...",0.0,1.0,0.0


In [11]:
# drop columns of scratch that CS column value is 99
scratch = scratch[scratch["CS"] != 99]
scratch = scratch[scratch["CS"] != 88]


In [12]:
# # plots geodataframe on mapbox map with hover.
# fig = px.choropleth_mapbox(allhfic, geojson=allhfic.geometry, locations=allhfic.index, color="CS", hover_name="CS", hover_data=["CS"], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [13]:
# # plotting mini df
# fig = px.choropleth_mapbox(scratch, geojson=scratch.geometry, locations=scratch.index, color="CS", hover_name="CS", hover_data=["CS"], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [14]:
eminidf = minidf
eminidf = eminidf[minidf["CS"] != 99]
eminidf = eminidf[minidf["CS"] != 88]

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [15]:
# print maximum value of scratch
print(scratch["CS"].max())


4.0


In [16]:
# # plots geodataframe on mapbox map with hover.
# fig = px.choropleth_mapbox(eminidf, geojson=eminidf.geometry, locations=eminidf.index, color="CS", hover_name="CS", hover_data=["CS"], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [17]:
# sort join_left_df by CS
lhz_join_full = lhz.sjoin(minidf, how="left")
lhz_join = lhz_join_full[lhz_join_full["CS"].notnull()]
lhz_join

,OBJECTID,FNID,EFF_YEAR,COUNTRY,LZNUM,LZCODE,LZNAMEEN,LZNAMEFR,LZNAMESP,LZNAMEPT,CLASS,LZSZCODE,LZTYPE,MAINCROPS,MAINLSTOCK,geometry,index_right,CS,HA0
31,61,CF2012L101,2012,CF,1,CF01,"Southeast cassava, peanuts, and maize","Sud-est manioc, arachide et maïs",None,None,AG12,None,None,None,None,"POLYGON ((26.48070 6.22050, 26.48873 6.21450, ...",2.0,2.0,0.0
31,61,CF2012L101,2012,CF,1,CF01,"Southeast cassava, peanuts, and maize","Sud-est manioc, arachide et maïs",None,None,AG12,None,None,None,None,"POLYGON ((26.48070 6.22050, 26.48873 6.21450, ...",4.0,3.0,0.0
32,62,CF2012L102,2012,CF,2,CF02,Northeast hunting and food crops,Nord-est chasse et vivriers,None,None,SH01,None,None,None,None,"POLYGON ((23.63402 9.46908, 23.64386 9.44569, ...",6.0,4.0,0.0
32,62,CF2012L102,2012,CF,2,CF02,Northeast hunting and food crops,Nord-est chasse et vivriers,None,None,SH01,None,None,None,None,"POLYGON ((23.63402 9.46908, 23.64386 9.44569, ...",0.0,1.0,0.0
32,62,CF2012L102,2012,CF,2,CF02,Northeast hunting and food crops,Nord-est chasse et vivriers,None,None,SH01,None,None,None,None,"POLYGON ((23.63402 9.46908, 23.64386 9.44569, ...",2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0,TD2021L106,2021,TD,6,TD06,Eastern Rainfed Cereals and Market Gardening,Zone est: céréales pluviales et maraïchage,None,None,AG10,None,None,None,None,"POLYGON ((22.94006 15.10751, 22.92099 15.09343...",3.0,2.0,1.0
857,0,TD2021L106,2021,TD,6,TD06,Eastern Rainfed Cereals and Market Gardening,Zone est: céréales pluviales et maraïchage,None,None,AG10,None,None,None,None,"POLYGON ((22.94006 15.10751, 22.92099 15.09343...",2.0,2.0,0.0
860,0,TD2021L104,2021,TD,4,TD04,Southeast Flood Retreat and Gum Arabic,Zone sud-est: cultures de décrue et gomme arab...,None,None,AN01,None,None,None,None,"POLYGON ((22.47709 12.01926, 22.47845 12.01923...",0.0,1.0,0.0
860,0,TD2021L104,2021,TD,4,TD04,Southeast Flood Retreat and Gum Arabic,Zone sud-est: cultures de décrue et gomme arab...,None,None,AN01,None,None,None,None,"POLYGON ((22.47709 12.01926, 22.47845 12.01923...",3.0,2.0,1.0


In [18]:
# drop columns of df that CS column value is 99
lhz_join = lhz_join[lhz_join["CS"] != 99]
lhz_join = lhz_join[lhz_join["CS"] != 88]

In [19]:
# # plots geodataframe on mapbox map with hover.
# fig = px.choropleth_mapbox(lhz_join, geojson=lhz_join.geometry, locations=lhz_join.index, color="CS", hover_name="CS", hover_data=["CS"], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [20]:
# save lhz_join to shapefile
lhz_join.to_file("./data/join_experiments/lhz.shp")
scratch.to_file("./data/join_experiments/adm2.shp")
eminidf.to_file("./data/join_experiments/original.shp")


/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_86459/1639181881.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lhz_join.to_file("./data/join_experiments/lhz.shp")
/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_86459/1639181881.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  scratch.to_file("./data/join_experiments/adm2.shp")
